<a href="https://colab.research.google.com/github/rodrigoamorimml/NaturalLanguageProcessing/blob/master/ToxicCommentClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this notebook it'll be using  the Kaggle Dataset Jig-Saw Toxic Comment for classification task, where can be found at: [Toxic Comment Dataset](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)

#### This dataset contains  differents types of toxic comments and the task it's to classify this comments in six possibles categories. Since a  single comment can be placed in more than one category, then it'll be used 6 binary classification to complete this task.

#### In addition it will be used different types of architeture of neural networks to compare their performance and accuracy in this dataset, like a feed foward neural network (MLP), ConvNet and a Reccurent Neural Network.

#### Besides that, for the pre processing stage it will be used a Pre-trained Word Embedding Vector that can be download at: [Glove Embedding Vector](http://nlp.stanford.edu/data/glove.6B.zip)

In [6]:
# Import Libraries

import os, sys

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Embedding, LSTM, SimpleRNN, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

1 - Load The data

In [7]:
# First, using the link above, we download the jig-saw dataset, then we can open using the pandas dataframe :

from google.colab import files

upload = files.upload()


Saving train.csv to train.csv


In [8]:
data = pd.read_csv('train.csv')
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [9]:
# The dataset contains 159571 rows (comments) and 8 columns, where the second column represents the toxic comments
## and the next 6 columns represents the possible labels classification. So we will be using the comment text as our input
## and the six categories as our labels

data.shape

(159571, 8)

In [10]:
# First we check if has any Nan Values
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [12]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']

X = data['comment_text'].values # get as numpy array
y = data[labels].values

print(f'Shape of Input Data: {X.shape}')
print(f'Shape of Output Data: {y.shape}')

Shape of Input Data: (159571,)
Shape of Output Data: (159571, 6)


**2 - Embedding Layer**

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
import zipfile

with zipfile.ZipFile('glove.6B.zip') as f:
  f.extractall()